In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20220919'
and yyyymmdd <= '20220925'
and city_name in ('Delhi')
and service_obj_service_name = 'Link'
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create a copy of the df retrived from presto
df = df_copy.copy()

In [5]:
#View the dataset
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,6328d2b249a7d319a067bfaf,0.939,883da11405fffff,28.614737,77.218639,Delhi,883da11407fffff,28.616028,77.225113,020602,02,0200,Link,02:00,2,20220920,customerCancelled,Asked to cancel & take offline ride
1,6328d6ea49a7d319a067c0b0,1.077,883da11183fffff,28.514357,77.106842,Delhi,883da1118dfffff,28.509411,77.082039,022402,02,0215,Link,02:00,2,20220920,dropped,None
2,6328d9cc49a7d319a067c138,2.340,883da10285fffff,28.515927,77.224626,Delhi,883da1153dfffff,28.558369,77.205650,023620,02,0230,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere
3,6328dc1d49a7d319a067c1b4,0.026,883da115d5fffff,28.588758,77.255295,Delhi,883da115d5fffff,28.588799,77.255203,024613,02,0245,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere
4,6328d92549a7d319a067c11a,0.010,883da1131bfffff,28.570454,77.018478,Delhi,883da112adfffff,28.587429,77.069824,023333,02,0230,Link,02:59,2,20220920,dropped,None


In [6]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date
0,6328d2b249a7d319a067bfaf,0.939,883da11405fffff,28.614737,77.218639,Delhi,883da11407fffff,28.616028,77.225113,020602,02,0200,Link,02:00,2,20220920,customerCancelled,Asked to cancel & take offline ride,02,06,20
1,6328d6ea49a7d319a067c0b0,1.077,883da11183fffff,28.514357,77.106842,Delhi,883da1118dfffff,28.509411,77.082039,022402,02,0215,Link,02:00,2,20220920,dropped,None,02,24,20
2,6328d9cc49a7d319a067c138,2.340,883da10285fffff,28.515927,77.224626,Delhi,883da1153dfffff,28.558369,77.205650,023620,02,0230,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,20,36,20
3,6328dc1d49a7d319a067c1b4,0.026,883da115d5fffff,28.588758,77.255295,Delhi,883da115d5fffff,28.588799,77.255203,024613,02,0245,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,13,46,20
4,6328d92549a7d319a067c11a,0.010,883da1131bfffff,28.570454,77.018478,Delhi,883da112adfffff,28.587429,77.069824,023333,02,0230,Link,02:59,2,20220920,dropped,None,33,33,20


In [7]:
#Create 15-sec batches and 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count
0,6328d2b249a7d319a067bfaf,0.939,883da11405fffff,28.614737,77.218639,Delhi,883da11407fffff,28.616028,77.225113,020602,02,0200,Link,02:00,2,20220920,customerCancelled,Asked to cancel & take offline ride,2.0,6.0,20,0-20,1
1,6328d6ea49a7d319a067c0b0,1.077,883da11183fffff,28.514357,77.106842,Delhi,883da1118dfffff,28.509411,77.082039,022402,02,0215,Link,02:00,2,20220920,dropped,None,2.0,24.0,20,0-20,1
2,6328d9cc49a7d319a067c138,2.340,883da10285fffff,28.515927,77.224626,Delhi,883da1153dfffff,28.558369,77.205650,023620,02,0230,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,20.0,36.0,20,0-20,1
3,6328dc1d49a7d319a067c1b4,0.026,883da115d5fffff,28.588758,77.255295,Delhi,883da115d5fffff,28.588799,77.255203,024613,02,0245,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,13.0,46.0,20,0-20,1
4,6328d92549a7d319a067c11a,0.010,883da1131bfffff,28.570454,77.018478,Delhi,883da112adfffff,28.587429,77.069824,023333,02,0230,Link,02:59,2,20220920,dropped,None,33.0,33.0,20,20-40,1


In [8]:
#Add column to identify the hexes
delhi_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone1.csv')
delhi_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone2.csv')

delhi_zone1_hexes = list(delhi_zone1['customer_location_hex_8'].unique())
delhi_zone2_hexes = list(delhi_zone2['customer_location_hex_8'].unique())

df['zone_name'] = np.where(df['customer_location_hex_8'].isin(delhi_zone1_hexes), 'zone1',
     np.where(df['customer_location_hex_8'].isin(delhi_zone2_hexes), 'zone2', 'rest_of_delhi'))
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count,zone_name
0,6328d2b249a7d319a067bfaf,0.939,883da11405fffff,28.614737,77.218639,Delhi,883da11407fffff,28.616028,77.225113,020602,02,0200,Link,02:00,2,20220920,customerCancelled,Asked to cancel & take offline ride,2.0,6.0,20,0-20,1,rest_of_delhi
1,6328d6ea49a7d319a067c0b0,1.077,883da11183fffff,28.514357,77.106842,Delhi,883da1118dfffff,28.509411,77.082039,022402,02,0215,Link,02:00,2,20220920,dropped,None,2.0,24.0,20,0-20,1,rest_of_delhi
2,6328d9cc49a7d319a067c138,2.340,883da10285fffff,28.515927,77.224626,Delhi,883da1153dfffff,28.558369,77.205650,023620,02,0230,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,20.0,36.0,20,0-20,1,rest_of_delhi
3,6328dc1d49a7d319a067c1b4,0.026,883da115d5fffff,28.588758,77.255295,Delhi,883da115d5fffff,28.588799,77.255203,024613,02,0245,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,13.0,46.0,20,0-20,1,rest_of_delhi
4,6328d92549a7d319a067c11a,0.010,883da1131bfffff,28.570454,77.018478,Delhi,883da112adfffff,28.587429,77.069824,023333,02,0230,Link,02:59,2,20220920,dropped,None,33.0,33.0,20,20-40,1,rest_of_delhi


In [9]:
#Create a column to identify OCARA, COBRA and COBRM
df['new_cancel_reason'] = np.where(df['order_status'] == 'customerCancelled', np.where(df['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
    np.where(df['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), df['order_status'])
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,20_sec_batch,count,zone_name,new_cancel_reason
0,6328d2b249a7d319a067bfaf,0.939,883da11405fffff,28.614737,77.218639,Delhi,883da11407fffff,28.616028,77.225113,020602,02,0200,Link,02:00,2,20220920,customerCancelled,Asked to cancel & take offline ride,2.0,6.0,20,0-20,1,rest_of_delhi,OCARA
1,6328d6ea49a7d319a067c0b0,1.077,883da11183fffff,28.514357,77.106842,Delhi,883da1118dfffff,28.509411,77.082039,022402,02,0215,Link,02:00,2,20220920,dropped,None,2.0,24.0,20,0-20,1,rest_of_delhi,dropped
2,6328d9cc49a7d319a067c138,2.340,883da10285fffff,28.515927,77.224626,Delhi,883da1153dfffff,28.558369,77.205650,023620,02,0230,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,20.0,36.0,20,0-20,1,rest_of_delhi,OCARA
3,6328dc1d49a7d319a067c1b4,0.026,883da115d5fffff,28.588758,77.255295,Delhi,883da115d5fffff,28.588799,77.255203,024613,02,0245,Link,02:59,2,20220920,customerCancelled,Found better price elsewhere,13.0,46.0,20,0-20,1,rest_of_delhi,OCARA
4,6328d92549a7d319a067c11a,0.010,883da1131bfffff,28.570454,77.018478,Delhi,883da112adfffff,28.587429,77.069824,023333,02,0230,Link,02:59,2,20220920,dropped,None,33.0,33.0,20,20-40,1,rest_of_delhi,dropped


In [10]:
#Filter the df on the 3 zones
df_zone1 = df[df['zone_name'] == 'zone1']
df_zone2 = df[df['zone_name'] == 'zone2']
df_zone_rest = df[df['zone_name'] == 'rest_of_delhi']

In [11]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone1
mistiming_pivot_zone1 = pd.pivot_table(data = df_zone1, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_zone1['total'] = mistiming_pivot_zone1[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_zone1['dropped_%'] = round(mistiming_pivot_zone1['dropped']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['COBRA_%'] = round(mistiming_pivot_zone1['COBRA']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['COBRM_%'] = round(mistiming_pivot_zone1['COBRM']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['OCARA_%'] = round(mistiming_pivot_zone1['OCARA']/mistiming_pivot_zone1['total'], 4)
mistiming_pivot_zone1['expiry_%'] = round(mistiming_pivot_zone1['expired']/mistiming_pivot_zone1['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_zone1 = mistiming_pivot_zone1.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_zone1.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.475893,0.146577,0.0,0.377529,0.0
1,19,01,0.485044,0.205128,0.0,0.309828,0.0
2,19,02,0.556452,0.104839,0.0,0.338710,0.0
3,19,03,0.406250,0.121527,0.0,0.472223,0.0
4,19,04,0.370000,0.110000,0.0,0.520000,0.0


In [13]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone2
mistiming_pivot_zone2 = pd.pivot_table(data = df_zone2, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_zone2['total'] = mistiming_pivot_zone2[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_zone2['dropped_%'] = round(mistiming_pivot_zone2['dropped']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['COBRA_%'] = round(mistiming_pivot_zone2['COBRA']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['COBRM_%'] = round(mistiming_pivot_zone2['COBRM']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['OCARA_%'] = round(mistiming_pivot_zone2['OCARA']/mistiming_pivot_zone2['total'], 4)
mistiming_pivot_zone2['expiry_%'] = round(mistiming_pivot_zone2['expired']/mistiming_pivot_zone2['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_zone2 = mistiming_pivot_zone2.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_zone2.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.477273,0.102273,0.0,0.386364,0.0
1,19,01,0.550000,0.050000,0.0,0.400000,0.0
2,19,02,0.392857,0.011905,0.0,0.595238,0.0
3,19,03,0.458333,0.104167,0.0,0.437500,0.0
4,19,04,0.521739,0.043478,0.0,0.434783,0.0


In [15]:
#Check the end order state for all orders occurring in a 20_sec_batch for zone2
mistiming_pivot_rest = pd.pivot_table(data = df_zone_rest, index = ['date', 'hour', 'minute', '20_sec_batch'], columns = 'new_cancel_reason', values = 'count',
aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Get the percentage for all order types
mistiming_pivot_rest['total'] = mistiming_pivot_rest[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expired', 'new']].sum(axis = 1)
mistiming_pivot_rest['dropped_%'] = round(mistiming_pivot_rest['dropped']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['COBRA_%'] = round(mistiming_pivot_rest['COBRA']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['COBRM_%'] = round(mistiming_pivot_rest['COBRM']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['OCARA_%'] = round(mistiming_pivot_rest['OCARA']/mistiming_pivot_rest['total'], 4)
mistiming_pivot_rest['expiry_%'] = round(mistiming_pivot_rest['expired']/mistiming_pivot_rest['total'], 4)

#Get the average of each percentage by hour
mistiming_grouped_rest = mistiming_pivot_rest.groupby(['date', 'hour'])['dropped_%', 'COBRA_%', 'COBRM_%', 'OCARA_%', 'expiry_%'].mean().reset_index()
mistiming_grouped_rest.head()

,date,hour,dropped_%,COBRA_%,COBRM_%,OCARA_%,expiry_%
0,19,00,0.377804,0.209442,0.005809,0.370600,0.021656
1,19,01,0.356195,0.200846,0.009958,0.404811,0.022549
2,19,02,0.317978,0.186680,0.019899,0.439884,0.035555
3,19,03,0.292154,0.188574,0.010745,0.468353,0.040171
4,19,04,0.387199,0.143851,0.015448,0.399579,0.049018


In [17]:
#Save the results
mistiming_grouped_zone1.to_csv('mistiming_delhi_zone1.csv', index = False)
mistiming_grouped_zone2.to_csv('mistiming_delhi_zone2.csv', index = False)
mistiming_grouped_rest.to_csv('mistiming_delhi_rest.csv', index = False)